In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df_train['Train_data'] = 1
df_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
df_test['Train_data'] = 0
df_test['Transported'] = 'Test'


all_data = pd.concat([df_train, df_test])
#all_data = df_train
removecolumns = []


In [ ]:
#Change Homeplante to numerical values
#all_data['HomePlanet'].value_counts()
all_data['HomePlanet'].replace('Earth', 1, inplace=True)
all_data['HomePlanet'].replace('Europa', 2, inplace=True)
all_data['HomePlanet'].replace('Mars', 3, inplace=True)


In [ ]:
#Change CryoSleep to numerical values
#all_data['CryoSleep'].value_counts()
all_data['CryoSleep'].replace(True, 1, inplace=True)
all_data['CryoSleep'].replace(False, 0, inplace=True)

In [ ]:
#Split cabin to fist letter to know the category
#Change Cabin_split values to numerical values
#all_data['Cabin'].value_counts()
all_data['Cabin_split'] = all_data['Cabin'].apply(lambda x: str(x).split('/')[0])

def con_alpha_num(x):
    alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','nan']
    return alpha.index(str.lower(x))

all_data['Cabin_split'] = all_data['Cabin_split'].apply(lambda x: con_alpha_num(x))
removecolumns.append('Cabin')

In [ ]:
#Change Destination values to numerical values
#all_data['Destination'].value_counts()

all_data['Destination'].replace('TRAPPIST-1e', 11, inplace=True)
all_data['Destination'].replace('55 Cancri e', 22, inplace=True)
all_data['Destination'].replace('PSO J318.5-22', 33, inplace=True)

In [ ]:
#Change VIP values to numerical values
#all_data['VIP'].value_counts()
#can be removed Not enough segregation to be used for classification
removecolumns.append('VIP')
#Change RoomService values to numerical values
#all_data['RoomService'].value_counts()
#can be removed Not enough segregation to be used for classification
removecolumns.append('RoomService')
#Change RoomService values to numerical values
#all_data['FoodCourt'].value_counts()
#can be removed Not enough segregation to be used for classification
removecolumns.append('FoodCourt')
#Change ShoppingMall values to numerical values
#all_data['ShoppingMall'].value_counts()
#can be removed Not enough segregation to be used for classification
removecolumns.append('ShoppingMall')
#Change Spa values to numerical values
#all_data['Spa'].value_counts()
#can be removed Not enough segregation to be used for classification
removecolumns.append('Spa')
#Change VRDeck values to numerical values
#all_data['VRDeck'].value_counts()
#can be removed Not enough segregation to be used for classification
#removecolumns.append('VRDeck')
#Change Name values to numerical values
#all_data['Name'].value_counts()
#can be removed Not enough segregation to be used for classification
removecolumns.append('Name')

In [ ]:
#Check the null values and replace or drop records

all_data['Age'].fillna(all_data['Age'].mean(), inplace=True)
all_data['Transported'].replace(True, 1, inplace=True)
all_data['Transported'].replace(False, 0, inplace=True)
#df_train_features.isnull().sum()
all_data.dropna(inplace=True)
all_data.drop(removecolumns, axis=1, inplace=True)

all_data_features = all_data[all_data['Train_data'] == 1]


In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split

feature_set = all_data_features[['HomePlanet', 'CryoSleep', 'Destination', 'Age','Cabin_split']]
Result_set = all_data_features['Transported']

x_train, x_test, y_train, y_test = train_test_split(feature_set, Result_set, random_state=0)
y_train = y_train.astype('int')
y_test = y_test.astype('int')
Result_set = Result_set.astype('int')

x_test = all_data[all_data['Train_data'] == 0]
x_test = x_test[['HomePlanet', 'CryoSleep', 'Destination', 'Age','Cabin_split']]

dctree = tree.DecisionTreeClassifier()
dctree.fit(x_train, y_train)
#dctree.score(x_test, y_test)
y_pred = dctree.predict(x_test)


In [ ]:
submission_df = pd.DataFrame()
submission_df["Transported"] = y_pred
submission_df.to_csv('submission.csv', index=False)